# Distributed

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lukeconibear/intro_ml/blob/main/docs/05_distributed.ipynb)

In [ ]:
# if you're using colab, then install the required modules
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    pass

Distributing training over multiple devices generally uses either:

- [Data parallelism](https://developers.google.com/machine-learning/glossary/#data-parallelism)
    - Single model copied to multiple devices.
    - Split data over multiple devices.
    - Useful for big data.
- [Model parallelism](https://developers.google.com/machine-learning/glossary/#model-parallelism)
    - Split model over multiple devices.
    - Single data copied to multiple devices.
    - Useful for big models.
    
This lesson focuses on data parallelism.

The distributed training can also be:

- Synchronous, where the training steps are synced across the workers.
- Asynchronous, where the training steps are not strictly synced across the workers.

## [TensorFlow](https://www.tensorflow.org/guide/distributed_training)

You can use `tf.distribute.Strategy` to distribute models and training over multiple machines with minimal code changes.

For example, there is [MirroredStrategy](https://www.tensorflow.org/guide/distributed_training#mirroredstrategy) for multiple replicas on one machine and [MultiWorkerMirroredStrategy](https://www.tensorflow.org/guide/distributed_training#multiworkermirroredstrategy) for synchronous training of multiple workers across many machines.

Within the strategy scope, you build a compiled the model.

```python
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = build_compiled_model()
    
model.fit(...)
```

```{note}

For multiple workers, you'll need to set the [`TF_CONFIG`](https://www.tensorflow.org/guide/distributed_training#setting_up_the_tf_config_environment_variable). There is an example [here](https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras).

```

In [ ]:
tf_config = {
    "cluster": {"worker": ["localhost:12345", "localhost:23456"]},
    "task": {"type": "worker", "index": 0},
}

In [11]:
import json

In [ ]:
json.dumps(tf_config)

An [example for MNIST](https://www.tensorflow.org/tutorials/distribute/keras):

In [1]:
import tensorflow as tf

2022-03-24 18:09:09.661248: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-24 18:09:09.661266: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def build_compiled_model():
    inputs = tf.keras.Input(shape=(784,))
    x = tf.keras.layers.Dense(256, activation="relu")(inputs)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    outputs = tf.keras.layers.Dense(10)(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    )

    return model

In [3]:
def get_dataset():
    batch_size = 32 * strategy.num_replicas_in_sync
    num_val_samples = 10000

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    x_train = x_train.reshape(-1, 784).astype("float32") / 255
    x_test = x_test.reshape(-1, 784).astype("float32") / 255
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")

    # reserve num_val_samples samples for validation
    x_val = x_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    x_train = x_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    
    return (
        tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size),
        tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size),
    )

Create a MirroredStrategy:

In [4]:
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1


2022-03-24 18:09:12.736716: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-24 18:09:12.736737: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-24 18:09:12.736759: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (UOL-LAP-5G6CZH3): /proc/driver/nvidia/version does not exist
2022-03-24 18:09:12.736948: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
with strategy.scope():
    model = build_compiled_model()

In [7]:
train_dataset, val_dataset, test_dataset = get_dataset()

```{tip}
The default [auto-sharding](https://www.tensorflow.org/api_docs/python/tf/data/experimental/AutoShardPolicy) by `FILE` can cause warning messages. Instead:

- Either disable auto-sharding using: `tf.data.experimental.AutoShardPolicy.OFF`  
- Or auto-shard by data: `tf.data.experimental.AutoShardPolicy.DATA`
```

In [8]:
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA

train_dataset = train_dataset.with_options(options)
val_dataset = val_dataset.with_options(options)
test_dataset = test_dataset.with_options(options)

In [13]:
history = model.fit(train_dataset, epochs=2, validation_data=val_dataset, verbose=False)

2022-03-24 18:40:36.770383: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


In [14]:
history.history

{'loss': [0.03628438338637352, 0.02810976840555668],
 'sparse_categorical_accuracy': [0.9881200194358826, 0.990559995174408],
 'val_loss': [0.09779918193817139, 0.10384755581617355],
 'val_sparse_categorical_accuracy': [0.9761000275611877, 0.9758999943733215]}

In [10]:
model.evaluate(test_dataset)

313/313 [==============================] - 0s 871us/step - loss: 0.0892 - sparse_categorical_accuracy: 0.9724


[0.0892195999622345, 0.9724000096321106]

Written up here:

[`tensorflow_mnist_example.py`](https://github.com/lukeconibear/intro_ml/blob/main/docs/distributed/tensorflow_mnist_example.py)


## Fault tolerance

Use Model Checkpointing (saw in the previous lesson) to be able to recover a model from a previous epoch if the training fails.

In [ ]:
import os

# Prepare a directory to store all the checkpoints.
checkpoint_dir = f"{os.getcwd()}/models/checkpoints"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [ ]:
def make_or_restore_model():
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        return tf.keras.models.load_model(latest_checkpoint)
    print("Creating a new model")
    return get_compiled_model()

In [ ]:
def run_training(epochs=1):
    # Create a MirroredStrategy.
    strategy = tf.distribute.MirroredStrategy()

    # Open a strategy scope and create/restore the model
    with strategy.scope():
        model = make_or_restore_model()

    callbacks = [
        # This callback saves a SavedModel every epoch
        # We include the current epoch in the folder name.
        tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_dir + "/checkpoint-{epoch}", save_freq="epoch"
        )
    ]
    model.fit(
        train_dataset,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=val_dataset,
        verbose=2,
    )

In [ ]:
# # Running the first time creates the model
# run_training(epochs=1)

In [ ]:
# # Calling the same function again will resume from where we left off
# run_training(epochs=1)

### Dataset caching

Add data caching to an example

and

### Prefetch data

Examples of how to distribute deep learning on a High Performance Computer (HPC).

## Install Python environments

First, install the Python environments for the required HPC.

## ARC4

### Miniconda installer
```bash
# download miniconda (x86_64 for ARC4)
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh

# run miniconda, read terms, and set path
. Miniconda3-latest-Linux-x86_64.sh
```

### Conda environment

#### Clone pre-created environments

```bash
# clone - tensorflow 2.7.0 and ray
conda env create --file tf_ray_arc4.yml

# clone - pytorch 1.10 and ray
module load gnu/8.3.0
conda env create --file pytorch_ray_arc4.yml
```

#### Create your own

```bash
# create new - tensorflow 2.7.0 and ray
conda create -n tf_ray_arc4 -c conda-forge python==3.9.* cudatoolkit==11.2.* cudnn==8.1.*
conda activate tf_ray_arc4
pip install -U pip
pip install tensorflow==2.7.0
pip install -U ray
pip install -U ray[tune]

# create new - pytorch (1.10) and ray
module load gnu/8.3.0
conda create -n pytorch_ray_arc4 pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch
conda activate pytorch_ray_arc4
pip install -U ray
pip install -U ray[tune]

# create new - jax
conda create -n jax python=3.8 cudatoolkit=11.2 cudatoolkit-dev=11.2 cudnn=8.2
conda activate jax
pip install -U jax
pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_releases.html
```

## Bede

### Miniconda installer
```bash
# Replace <project> with your project code
export DIR=/nobackup/projects/<project>/$USER

# download miniconda (ppc64le for Bede's hardware, not x86_64 as for ARC4)
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-ppc64le.sh
 
# run miniconda
sh Miniconda3-latest-Linux-ppc64le.sh -b -p $DIR/miniconda
source miniconda/bin/activate
 
# update conda and set channels
conda update conda -y
conda config --prepend channels conda-forge
conda config --prepend channels https://public.dhe.ibm.com/ibmdl/export/pub/software/server/ibm-ai/conda/
conda config --prepend channels https://opence.mit.edu
```

This is what my `~/.condarc` ends up as:
```bash
channel_priority: flexible
channels:
  - https://opence.mit.edu
  - https://public.dhe.ibm.com/ibmdl/export/pub/software/server/ibm-ai/conda/
  - conda-forge
  - defaults
```

### Conda environment

#### Clone pre-created environments

```bash
# clone - tensorflow 2.7.0 and ray
conda env create --file tf_bede.yml

# clone - pytorch 1.10 and ray
conda env create --file pytorch_bede.yml

# clone - pytorch 1.9.0, cuda 10.2, and pytorch_geometric 2.0.3
module load gcc # require this for some of the libraries
conda env create --file pytorch_geometric_bede.yml
```

#### Create your own

```bash
# create an environment for pytorch
conda create -n pytorch pytorch torchvision cudatoolkit=10.2
 
# create an environment for tensorflow
conda create -n tf tensorflow

# create an environment for pytorch geometric
module load gcc
conda create -n pytorch_geometric pytorch cudatoolkit=10.2
conda activate pytorch_geometric

pip install torch-scatter
pip install torch-sparse
pip install torch-geometric
pip install torch-cluster
```

## JADE-2

### Miniconda installer

```bash
...
```

### Conda environment

#### Clone pre-created environments

```bash
...
```

#### Create your own

```bash
...
```

## Jupyter Notebook to HPC

It's preferable to use a static job on the HPC. To do this, you could test out different ideas locally in a Jupyter Notebook, then when ready convert this to an executable script (`.py`) and move it over. 

...

## [Ray Train](https://docs.ray.io/en/latest/train/train.html)

These examples use Ray Train in a static job on a HPC.
Ray handles most of the complexity of distributing the work, with minimal changes to your [TensorFlow](https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras) or [PyTorch](https://pytorch.org/tutorials/beginner/dist_overview.html) code.

- Python script examples:
  - TensorFlow
    - MNIST: [`tensorflow_ray_mnist_example.py`](https://github.com/lukeconibear/intro_ml/blob/main/docs/distributed/tensorflow_ray_mnist_example.py).  
  - PyTorch
    - Fashion MNIST: [`pytorch_ray_train_fashion_mnist_example.py`](https://github.com/lukeconibear/intro_ml/blob/main/docs/distributed/pytorch_ray_train_fashion_mnist_example.py).  
- Then submit the job to HPC (choose one and update the Python script within it):
  - [ARC4](https://arcdocs.leeds.ac.uk/systems/arc4.html) (SGE)  
    - CPU: [`distributed_ml_on_arc4_cpu.bash`](https://github.com/lukeconibear/intro_ml/blob/main/docs/distributed/distributed_ml_on_arc4_cpu.bash).  
    - GPU: [`distributed_ml_on_arc4_gpu.bash`](https://github.com/lukeconibear/intro_ml/blob/main/docs/distributed/distributed_ml_on_arc4_gpu.bash).  
  - [Bede](https://bede-documentation.readthedocs.io/en/latest/) (SLURM)
    - GPU: [`distributed_ml_on_bede.bash`](https://github.com/lukeconibear/intro_ml/blob/main/docs/distributed/distributed_ml_on_bede.bash).  
  - [JADE-2](http://docs.jade.ac.uk/en/latest/index.html) (SLURM)
    - GPU: ...



## Exercises

```{admonition} Exercise 1

...

```

## {ref}`Solutions <distributed>`

## Key Points

```{important}

- [x] _..._

```

## Further information

### Good practices

- Batch the dataset with the global batch size e.g., for 8 devices each capable of a btach of 64 use the global batch size of 512 (= 8 * 64).  
- ...

### Other options

- [Horovod](https://horovod.ai/)
- [DeepSpeed](https://www.deepspeed.ai/)
 
### Resources

- ...